## **<h3 align="center"> Deep Learning - Project </h3>**
# **<h3 align="center"> Phylum Arthropoda - Steven</h3>**
**Group 4 members:**<br>
Alexandra Pinto - 20211599@novaims.unl.pt - 20211599<br>
Steven Carlson - 20240554@novaims.unl.pt - 20240554<br>
Sven Goerdes - 20240503@novaims.unl.pt - 20240503<br>
Tim Straub - 20240505@novaims.unl.pt - 20240505<br>
Zofia Wojcik  - 20240654@novaims.unl.pt - 20240654<br>

# Table of Contents
* [1. Introduction](#intro)
* [2. Setup](#setup)
* [3. Data Loading](#dataloading)
* [4. Image Preprocessing](#imagepreprocessing)
* [5. Neural Networks Models](#nnmodels)



# 1. Introduction <a class="anchor" id="intro"></a>

In this second notebook, we will preprocess images from the **Arthropoda** phylum and develop a deep learning model to accurately classify them at the family level.

# 2. Setup <a class="anchor" id="setup"></a>
In this section, we will import the necessary libraries that will be used throughout the notebook. These libraries will help with data handling and image processing.

In [36]:
# Standard libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import zipfile
import seaborn as sns
import itertools
import random

# Libraries for image processing
from glob import glob
from PIL import Image


In [37]:
#Libraries from Keras / TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.utils import image_dataset_from_directory
from keras import optimizers
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


#import tensorflow_hub as hub

#Import pre-trained models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess




# 3. Data Loading <a class="anchor" id="dataloading"></a>

Let's open the train and test for Arthropoda Phylum.

In [38]:
# Load the DataFrame from the CSV file
arthropoda_train = pd.read_csv("/home/sacar/DeepLearning2425/train_test_splits/arthropoda_train.csv")
arthropoda_train.head(3)

,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,28260809,1065329,animalia,arthropoda,apidae,arthropoda_apidae/28260809_1065329_eol-full-si...
1,29945328,1077217,animalia,arthropoda,pseudophasmatidae,arthropoda_pseudophasmatidae/29945328_1077217_...
2,14644212,463474,animalia,arthropoda,formicidae,arthropoda_formicidae/14644212_463474_eol-full...


In [39]:
# Load the DataFrame from the CSV file
arthropoda_test = pd.read_csv("/home/sacar/DeepLearning2425/train_test_splits/arthropoda_test.csv")
arthropoda_test.head(3)

,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,28408206,1065346,animalia,arthropoda,apidae,arthropoda_apidae/28408206_1065346_eol-full-si...
1,28253620,1065348,animalia,arthropoda,apidae,arthropoda_apidae/28253620_1065348_eol-full-si...
2,21847584,1065348,animalia,arthropoda,apidae,arthropoda_apidae/21847584_1065348_eol-full-si...


# 4. Image Preprocessing <a class="anchor" id="imagepreprocessing"></a>

In [40]:
#Control Panel -- Choose one model to run!
is_resnet = 0
is_mobilenet = 1
is_efficientnet = 0
is_densenet = 0
is_inception = 0
is_convnext = 0

In [41]:
#Define preprocess and augmentation functions

#Function to preprocess the images
def process_image(file_path, label):
    image = tf.io.read_file(file_path) # Read the image file
    image = tf.image.decode_jpeg(image, channels=3) # Decode the JPEG image
    image = tf.image.resize(image, image_size) # Resize the image to the target size
    
    #CHANGE THIS LINE DEPENDING ON WHICH PRE-TRAINED MODEL IS BEING USED
    
    if is_resnet:
        image = resnet_preprocess(image)  # Apply ResNet50 preprocessing
        print("ResNet50 preprocessing applied")
    elif is_mobilenet: 
        image = mobilenet_preprocess(image)  # Apply MobileNetV2 preprocessing
        print
    elif is_efficientnet:
        image = efficientnet_preprocess(image)  # Apply EfficientNetB0 preprocessing
        print("EfficientNetB0 preprocessing applied")
    elif is_densenet:
        image = densenet_preprocess(image)  # Apply DenseNet121 preprocessing
        print("DenseNet121 preprocessing applied")
    elif is_inception:
        image = inception_preprocess(image)  # Apply InceptionV3 preprocessing
        print("InceptionV3 preprocessing applied")
    else:
        image = tf.cast(image, tf.float32) / 255.0  # Apply ConvNeXt / default preprocessing
        print("ConvNeXt preprocessing applied")
    
    return image, label


#Function to augment the images
def augment_image(image, label):

    #Randomly change brightness
    image = tf.image.random_brightness(image, max_delta=0.2)

    #Apply geometric augmentations
    image = geometric_augmentation_layers(image, training=True) # Apply geometric augmentations
    image = tf.clip_by_value(image, 0.0, 1.0)
    
    return image, label


# Geometric augmentations
geometric_augmentation_layers = tf.keras.Sequential(
    [
        # Randomly flip horizontally
        tf.keras.layers.RandomFlip("horizontal"),

        # Randomly rotate
        tf.keras.layers.RandomRotation(factor=0.12),

        # Random zoom
        tf.keras.layers.RandomZoom(height_factor=(-0.35, 0.35), # Corresponds to [0.8, 1.2] of original height
                                   width_factor=(-0.35, 0.35)), # Corresponds to [0.8, 1.2] of original width

        # Random shift
        tf.keras.layers.RandomTranslation(height_factor=0.20,
                                          width_factor=0.20),

        # Contrast
        tf.keras.layers.RandomContrast(factor=0.25),

    ],
    name="geometric_augmentations",
)


In [42]:
#Define some stuff
num_classes = arthropoda_train['family'].nunique() #number of classes = number of families
batch_size = 64
input_shape = (224, 224, 3)
image_size = (224, 224)
value_range = (0.0, 1.0)
num_classes = 17  

# Define callbacks
my_callbacks = [
callbacks.EarlyStopping(patience=10, restore_best_weights=True),
callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5),
callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)
]



In [43]:
root_dir = "/home/sacar/DeepLearning2425/rare_species"

#Get the file paths and labels for the training and test set
arthropoda_train['full_path'] = arthropoda_train['file_path'].apply(lambda x: os.path.normpath(os.path.join(root_dir, x)))
arthropoda_test['full_path'] = arthropoda_test['file_path'].apply(lambda x: os.path.normpath(os.path.join(root_dir, x)))

file_paths_train = arthropoda_train['full_path'].tolist()
labels_train = arthropoda_train['family'].tolist()

file_paths_test = arthropoda_test['full_path'].tolist()
labels_test = arthropoda_test['family'].tolist()

#Map the labels to integers
label_names = sorted(set(labels_train)) # Get the unique labels
label_to_index = {name: i for i, name in enumerate(label_names)} # Create a mapping from labels to integers
labels_train = [label_to_index[label] for label in labels_train]
labels_test = [label_to_index[label] for label in labels_test]

print(file_paths_train[:5])
print(labels_train[:5])
print(file_paths_test[:5])
print(labels_test[:5])

['/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/28260809_1065329_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_pseudophasmatidae/29945328_1077217_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_formicidae/14644212_463474_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_pseudophasmatidae/29945335_1077217_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_papilionidae/21035374_130548_eol-full-size-copy.jpg']
[0, 13, 5, 13, 10]
['/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/28408206_1065346_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/28253620_1065348_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/21847584_1065348_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_formicidae/14681521_403723_eol-full-size-copy.jpg', '/home/sacar/DeepLe

In [44]:
#Create the tensorflow datasets

#Load data
data = tf.data.Dataset.from_tensor_slices((file_paths_train, labels_train))
data = data.shuffle(buffer_size=len(file_paths_train), reshuffle_each_iteration=False, seed=42) # Shuffle the dataset

#Create train/val
train_size = int(0.8 * len(file_paths_train)) #80% for training, 20% for validation
train = data.take(train_size) # Take the first 80% for training
val = data.skip(train_size) # Skip the first 80% for validation

#Training preprocess pipeline
train = train.map(process_image, num_parallel_calls=tf.data.AUTOTUNE) # Map the function to the dataset
train = train.cache() # Cache the dataset for faster access
train = train.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE) # Map the function to the dataset
train = train.shuffle(buffer_size=1000, reshuffle_each_iteration=True, seed=42).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE) #shuffle and batch

#Validation preprocess pipeline
val = val.map(process_image, num_parallel_calls=tf.data.AUTOTUNE) # Map the function to the dataset
val = val.cache() # Cache the dataset for faster access
val = val.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE) #batch


#Test preprocess pipeline
test = tf.data.Dataset.from_tensor_slices((file_paths_test, labels_test))
test = test.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
test = test.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [45]:
for image, label in train.take(3):
    print("Image shape:", image.numpy().shape)
    print("Label:", label.numpy())

for image, label in test.take(3):
    print("Image shape:", image.numpy().shape)
    print("Label:", label.numpy())

Image shape: (32, 224, 224, 3)
Label: [ 0  8  5  5  2  8  0  5  4  0  5  5  5  6  0  1  3  5  3 11  6 10  1  3
  0  0  0  0 14  5 10  5]
Image shape: (32, 224, 224, 3)
Label: [11  5  5 12  7  5 15  2  5  0  6 13  5  6  1  5 16 12 14  5  5  4 13  0
  5 15  7  6  7  0 14  5]
Image shape: (32, 224, 224, 3)
Label: [16  0 11 13 13  1  5  3  6  3  5  8  1  5 11 10  6  6  5  2  9  5  0  5
  5  2  5  4 10  5  0  0]
Image shape: (32, 224, 224, 3)
Label: [ 0  0  0  5  9  0 16  5  5  5  0 16  6  0 11  0  0  5 13  5  5  8  5  0
 16  7  7  2  5  6  8  0]
Image shape: (32, 224, 224, 3)
Label: [ 5 12 15  0  5 14 12 11  8  0  5  6 12  0  6  0  3  5  5  1  5  5  7  0
  0  3 13  0  7  5  5  5]
Image shape: (32, 224, 224, 3)
Label: [13  2  5  9 14  5  6  0  5  5  7  5 15  0 15 10  4  0 14  1  5  5  5  5
  0  2  5 10  3  0 11  7]


2025-04-10 20:14:59.730339: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-04-10 20:14:59.740877: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# 5. Neural Network Models <a class="anchor" id="nnmodels"></a>

In [46]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [47]:
# Print GPU devices detected
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU(s) detected: {[gpu.name for gpu in gpus]}")
else:
    print("❌ No GPU detected by TensorFlow.")

✅ GPU(s) detected: ['/physical_device:GPU:0']


# Resnet50

In [48]:
if is_resnet == 1:
    # 1. Load the ResNet50 base model (without top classifier)
    base_model = ResNet50(
        weights='imagenet',          # Use ImageNet pretrained weights
        include_top=False,           # Exclude the default final dense layer
        input_shape=(224, 224, 3)    # Adjust to match your input images
    )

    # 2. Freeze the base model so we only train the classifier head
    base_model.trainable = False

    # 3. Build custom classification head
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)


    # 4. Create the full model
    model = models.Model(inputs=base_model.input, outputs=output)

    # 5. Compile the model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',  # ✅ Use sparse version for integer labels
        metrics=['accuracy']
    )


    # 6. Train the model
    history = model.fit(
        train,
        epochs=100,
        validation_data=val,
        callbacks=[EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)],
        verbose=1
    )

    # 7. Save the model
    model.save("resnet50_rare_species_model.h5")

    plot_filename = 'resnet50_rare_species_1.png' # Match naming convention
    plt.savefig(plot_filename)
    print(f"Training plot saved to {plot_filename}")
    
    # Plot training & validation accuracy
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()

    #Plot training & validation F1 score
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['f1_score'])
    plt.plot(history.history['val_f1_score'])
    plt.title('Model F1 Score')
    plt.ylabel('F1 Score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()



# MobileNetV2

In [49]:
'''
if is_mobilenet == 1:
    # Load base model without top layer
    base_model = MobileNetV2(
        input_shape=(224, 224, 3),  # Match your resized image shape
        include_top=False,         # Don't include the original classifier
        weights='imagenet'         # Use ImageNet-pretrained weights
    )

    # Define callbacks
    my_callbacks = [
    callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5),
    callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)
    ]


    #Phase 1: 
    # Freeze base model so we only train the new layers for now
    base_model.trainable = False

    # Build custom model
    model = models.Sequential([
        base_model,                                # Feature extractor
        layers.GlobalAveragePooling2D(),           # Pool over spatial dimensions
        layers.Dropout(0.5),                       # Regularization layer
        layers.Dense(256, activation='relu'),      # Fully connected layer
        layers.Dense(num_classes, activation='softmax')      # Output layer (5 mollusk families)
    ])

    # Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',  # Use sparse version for integer labels
        metrics=['accuracy'],
    )

    # Train the model
    history1 = model.fit(
        train,
        validation_data=val,
        callbacks = my_callbacks,
        epochs=100,           # Start small, increase if needed
        verbose=1
    )

    #Phase 2:
    # Unfreeze everything but the first 100 layers
    base_model.trainable = True
    for layer in base_model.layers[:100]:
        layer.trainable = False

    # Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',  # Use sparse version for integer labels
        metrics=['accuracy'],
    )

    # Train the model
    history2 = model.fit(
        train,
        validation_data=val,
        callbacks = my_callbacks,
        epochs=100,           # Start small, increase if needed
        verbose=1
    )

    # Save the model
    model.save("mobilenetv2_rare_species_model.h5")

    # Plot training & validation accuracy
    plt.figure(figsize=(12, 5))
    plt.plot(history2.history['accuracy'])
    plt.plot(history2.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()

    #Plot training & validation F1 score
    plt.figure(figsize=(12, 5))
    plt.plot(history2.history['f1_score'])
    plt.plot(history2.history['val_f1_score'])
    plt.title('Model F1 Score')
    plt.ylabel('F1 Score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()
    
    #Plot training & validation loss
    plt.figure(figsize=(12, 5))
    plt.plot(history2.history['loss'])
    plt.plot(history2.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()


    plot_filename = 'mobilenetV2_rare_species_1.png' # Match naming convention
    plt.savefig(plot_filename)
    print(f"Training plot saved to {plot_filename}")
    '''

'\nif is_mobilenet == 1:\n    # Load base model without top layer\n    base_model = MobileNetV2(\n        input_shape=(224, 224, 3),  # Match your resized image shape\n        include_top=False,         # Don\'t include the original classifier\n        weights=\'imagenet\'         # Use ImageNet-pretrained weights\n    )\n\n    # Define callbacks\n    my_callbacks = [\n    callbacks.EarlyStopping(patience=10, restore_best_weights=True),\n    callbacks.ReduceLROnPlateau(monitor=\'val_loss\', factor=0.5, patience=5),\n    callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)\n    ]\n\n\n    #Phase 1: \n    # Freeze base model so we only train the new layers for now\n    base_model.trainable = False\n\n    # Build custom model\n    model = models.Sequential([\n        base_model,                                # Feature extractor\n        layers.GlobalAveragePooling2D(),           # Pool over spatial dimensions\n        layers.Dropout(0.5),                       # Regularization

In [ ]:
def run_random_search(train, val, num_classes, n_trials=15, seed=42):

    random.seed(seed)

    # Hyperparameter search space
    dropout_rates = [0.5, 0.6, 0.7]
    dense_units_list = [64, 128]
    learning_rates = [1e-5, 5e-5, 1e-4]
    patience_values = [5, 7]
    freeze_until_layers = [100, 120, 140]
    optimizers_list = ['adam']

    # Generate all possible combinations
    all_combinations = list(itertools.product(
        dropout_rates,
        dense_units_list,
        learning_rates,
        patience_values,
        freeze_until_layers,
        optimizers_list
    ))

    # Randomly sample N trials
    sampled_combinations = random.sample(all_combinations, k=min(n_trials, len(all_combinations)))

    results = []

    for i, (dropout, units, lr, patience, freeze_until, opt_name) in enumerate(sampled_combinations):
        print(f"\n Trial {i+1}/{len(sampled_combinations)}")
        print(f"Dropout={dropout}, Units={units}, LR={lr}, Patience={patience}, FreezeUntil={freeze_until}")

        # Optimizer selection
        optimizer = optimizers.Adam(learning_rate=lr)

        # --- Phase 1 ---
        base_model = MobileNetV2(
            input_shape=(224, 224, 3),
            include_top=False,
            weights='imagenet'
        )
        base_model.trainable = False

        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dropout(dropout),
            layers.Dense(units, activation='relu'),
            layers.Dense(num_classes, activation='softmax')
        ])

        model.compile(
            optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        callbacks_list = [
            EarlyStopping(patience=patience, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=max(1, patience // 2)),
            ModelCheckpoint("temp_best_model.h5", save_best_only=True)
        ]

        history1 = model.fit(
            train,
            validation_data=val,
            epochs=30,
            callbacks=callbacks_list,
            verbose=0
        )

        # --- Phase 2 ---
        base_model.trainable = True
        for layer in base_model.layers[:freeze_until]:
            layer.trainable = False

        model.compile(
            optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        history2 = model.fit(
            train,
            validation_data=val,
            epochs=30,
            callbacks=callbacks_list,
            verbose=0
        )

        final_val_acc = history2.history['val_accuracy'][-1]
        results.append({
            'dropout': dropout,
            'dense_units': units,
            'learning_rate': lr,
            'patience': patience,
            'freeze_until': freeze_until,
            'optimizer': opt_name,
            'val_accuracy': final_val_acc
        })

    return pd.DataFrame(results)


In [51]:
def plot_random_search_results(results_df, top_n=10):
    sns.set(style="whitegrid")

    # -------------------------------
    # 1. Top N Configurations by Val Accuracy
    # -------------------------------
    top_configs = results_df.sort_values(by='val_accuracy', ascending=False).head(top_n)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(data=top_configs, x='val_accuracy', y=top_configs.index, hue='dropout')
    plt.title(f"Top {top_n} Hyperparameter Configs by Validation Accuracy")
    plt.xlabel("Validation Accuracy")
    plt.ylabel("Config Index")
    plt.legend(title="Dropout")
    plt.tight_layout()
    plt.show()

    # -------------------------------
    # 2. Strip plots: Accuracy vs Each Hyperparam
    # -------------------------------
    fig, axs = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle("Validation Accuracy vs Hyperparameters", fontsize=16)

    sns.stripplot(data=results_df, x='dropout', y='val_accuracy', ax=axs[0, 0])
    axs[0, 0].set_title("Dropout")

    sns.stripplot(data=results_df, x='dense_units', y='val_accuracy', ax=axs[0, 1])
    axs[0, 1].set_title("Dense Units")

    sns.stripplot(data=results_df, x='learning_rate', y='val_accuracy', ax=axs[0, 2])
    axs[0, 2].set_title("Learning Rate")

    sns.stripplot(data=results_df, x='patience', y='val_accuracy', ax=axs[1, 0])
    axs[1, 0].set_title("EarlyStopping Patience")

    sns.stripplot(data=results_df, x='freeze_until', y='val_accuracy', ax=axs[1, 1])
    axs[1, 1].set_title("Freeze Until Layer")

    axs[1, 2].axis('off')  # Empty slot

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    # -------------------------------
    # 3. Heatmap (e.g. Dropout vs Units)
    # -------------------------------
    pivot_table = results_df.pivot_table(
        values='val_accuracy',
        index='dropout',
        columns='dense_units',
        aggfunc='mean'
    )

    plt.figure(figsize=(8, 6))
    sns.heatmap(pivot_table, annot=True, fmt=".3f", cmap="viridis")
    plt.title("Heatmap: Dropout vs Dense Units (Val Accuracy)")
    plt.xlabel("Dense Units")
    plt.ylabel("Dropout Rate")
    plt.tight_layout()
    plt.show()


In [52]:
if is_mobilenet == 1:
    random_results = run_random_search(train, val, num_classes)
    random_results = random_results.sort_values(by='val_accuracy', ascending=False)
    random_results.to_csv("random_results.csv", index=False)
    print(random_results.head(10))
    plot_grid_search_results(results_df)




🔎 Trial 1/15
Dropout=0.7, Units=64, LR=5e-05, Patience=7, FreezeUntil=100


I0000 00:00:1744312507.976793   98264 service.cc:152] XLA service 0x7fa7e80112d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744312507.977064   98264 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-04-10 20:15:08.096112: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744312509.158690   98264 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1744312516.788084   98264 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


NotImplementedError: numpy() is only available when eager execution is enabled.

# EfficientNetB0

In [ ]:
if is_efficientnet == 1:
    # Load EfficientNetB0 as base model
    base_model = EfficientNetB0(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )

    # Freeze base model
    base_model.trainable = False

    # Build classifier on top
    model = models.Sequential([
        base_model,                                # Feature extractor
        layers.GlobalAveragePooling2D(),           # Pool features
        layers.Dropout(0.3),                       # Regularization
        layers.Dense(128, activation='relu'),      # Fully connected layer
        layers.Dense(num_classes, activation='softmax')  # Output layer
    ])

    # Compile the model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Print summary
    model.summary()

    # Train it
    history = model.fit(
        train,
        validation_data=val,
        epochs=100,         # Adjust as needed
        verbose=1
    )

    #Save the model
    model.save("efficientnet_rare_species_model.h5")
    
    plot_filename = 'efficientnet_rare_species_1.png' # Match naming convention
    plt.savefig(plot_filename)
    print(f"Training plot saved to {plot_filename}")

    # Plot training & validation accuracy
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()

    #Plot training & validation F1 score
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['f1_score'])
    plt.plot(history.history['val_f1_score'])
    plt.title('Model F1 Score')
    plt.ylabel('F1 Score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()


# DenseNet

In [ ]:
if is_densenet == 1:
    base_model = DenseNet121(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )

    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    history = model.fit(
        train,
        validation_data=val,
        epochs=100,
        verbose=1
    )

    # Save the model
    model.save("densenet_rare_species_model.h5")

    plot_filename = 'densenet_rare_species_1.png' # Match naming convention
    plt.savefig(plot_filename)
    print(f"Training plot saved to {plot_filename}")

    # Plot training & validation accuracy
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()

    #Plot training & validation F1 score
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['f1_score'])
    plt.plot(history.history['val_f1_score'])
    plt.title('Model F1 Score')
    plt.ylabel('F1 Score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()

# InceptionV3

In [ ]:
if is_inception == 1:
    base_model = InceptionV3(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )

    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    history = model.fit(
        train,
        validation_data=val,
        epochs=100,
        verbose=1
    )

    #Save the model
    model.save("inception_rare_species_model.h5")

    plot_filename = 'inceptionV3_rare_species_1.png' # Match naming convention
    plt.savefig(plot_filename)
    print(f"Training plot saved to {plot_filename}")

    # Plot training & validation accuracy
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()

    #Plot training & validation F1 score
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['f1_score'])
    plt.plot(history.history['val_f1_score'])
    plt.title('Model F1 Score')
    plt.ylabel('F1 Score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()

# Convnext

In [ ]:
if is_convnext == 1:
    base_model = tf.keras.Sequential([
        hub.KerasLayer(
            "https://tfhub.dev/sayakpaul/convnext_tiny_1k_224/1",
            trainable=False,
            input_shape=(224, 224, 3)
        )
    ])

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    history = model.fit(
        train,
        validation_data=val,
        epochs=100,
        verbose=1)
    
    #Save the model
    model.save("convnext_rare_species_model.h5")

    plot_filename = 'convnext_rare_species_1.png' # Match naming convention
    plt.savefig(plot_filename)
    print(f"Training plot saved to {plot_filename}")

    # Plot training & validation accuracy
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()

    #Plot training & validation F1 score
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['f1_score'])
    plt.plot(history.history['val_f1_score'])
    plt.title('Model F1 Score')
    plt.ylabel('F1 Score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'])
    plt.show()